# 🔧 Config Test Notebook

This notebook demonstrates how to:
- Load configs from the Delta config store
- Fetch specific keys
- Reload configs after updates
- Insert multiple configs to a dummy environment
- Delete all configs for a specific environment


In [0]:
# Setup Spark reference
spark = spark

# Set catalog, schema, and table
CATALOG = "arao"
SCHEMA = "aerodemo"
TABLE_NAME = f"{CATALOG}.{SCHEMA}.aerodemo_config_store"

print(f"✅ Using config table: {TABLE_NAME}")

In [0]:
# Read the entire table
df = spark.read.table(TABLE_NAME)
display(df)

In [0]:
# Define environment (e.g., 'dev', 'staging', 'prod')
env = "dev"

# Filter configs for that environment
env_df = df.filter(df.env == env)
display(env_df)

In [0]:
# Define which config key to fetch
target_key = "e2e_workflow_job_id"

# Retrieve the value
result = (
    df.filter((df.env == env) & (df.config_key == target_key))
      .select("config_value")
      .collect()
)

if result:
    config_value = result[0]["config_value"]
    print(f"✅ {env} → {target_key}: {config_value}")
else:
    print(f"⚠️ Config key '{target_key}' not found for env '{env}'")

In [0]:
# Convert all configs to a Python dictionary for the selected env
env_configs = {
    row["config_key"]: row["config_value"]
    for row in env_df.collect()
}

print(f"✅ Configs for env '{env}':\n{env_configs}")

In [0]:
# Import the method (assuming your repo is set up to include `.config` as a package)
from conf.config_reader import load_env_configs

# Get configs
env = "dev"
configs = load_env_configs(env)

print(f"✅ Loaded configs for {env}: {configs}")

# Access a specific config
job_id = configs.get("e2e_workflow_job_id")
print(f"🔑 e2e_workflow_job_id → {job_id}")

%md
## 📥 Load Configs for an Environment

In [0]:

from conf.config_reader import load_env_configs

env = "dev"
configs = load_env_configs(env)
print(f"✅ Loaded configs for {env}: {configs}")


%md
### 🔑 Fetch a specific config key

In [0]:

job_id = configs.get("e2e_workflow_job_id")
print(f"🔑 e2e_workflow_job_id → {job_id}")


%md
### ♻️ Reload configs after updates

In [0]:

# Imagine configs were updated elsewhere
configs = load_env_configs(env)
print(f"♻️ Reloaded configs: {configs}")

## 📦 Insert Multiple Configs to Dummy Environment

In [0]:

from pyspark.sql import Row

dummy_env = "dummy"
configs_to_insert = [
    (dummy_env, "key1", "value1"),
    (dummy_env, "key2", "value2"),
    (dummy_env, "key3", "value3")
]

df = spark.createDataFrame([Row(env=e, config_key=k, config_value=v) for e, k, v in configs_to_insert])
df.createOrReplaceTempView("temp_dummy_upserts")

TABLE_NAME = "arao.aerodemo.aerodemo_config_store"

spark.sql(f'''
MERGE INTO {TABLE_NAME} AS target
USING temp_dummy_upserts AS source
ON target.env = source.env AND target.config_key = source.config_key
WHEN MATCHED THEN UPDATE SET target.config_value = source.config_value
WHEN NOT MATCHED THEN INSERT (env, config_key, config_value) VALUES (source.env, source.config_key, source.config_value)
''')

print(f"✅ Inserted {len(configs_to_insert)} configs into {dummy_env} environment")


## 🗑️ Delete All Configs for an Environment

In [0]:

env_to_delete = "dummy"
spark.sql(f"DELETE FROM {TABLE_NAME} WHERE env = '{env_to_delete}'")
print(f"✅ Deleted all configs for environment: {env_to_delete}")
